# Exploratory Data Analysis and Data Cleaning

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime
from dateutil import parser
from sklearn.model_selection import train_test_split
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from sklearn.linear_model import LogisticRegression
%matplotlib inline
sns.set()

In [2]:
crashes = pd.read_csv('../Traffic_Crashes_-_Crashes.csv')

In [3]:
# Low_memory=False, because it has mixed types of data in the same column/too many rows
people = pd.read_csv('../Traffic_Crashes_-_People.csv', low_memory=False)

In [4]:
vehicles = pd.read_csv('../Traffic_Crashes_-_Vehicles.csv', low_memory=False)

In [5]:
def howmanyunique(data):
    tempo = []
    nombre = []
    for x in data.columns:
        tempo.append(len(data[x].value_counts().unique()))
        nombre.append(x)
    print(tempo)
    print('')
    print(nombre)

In [6]:
def howmanynan(data):
    print(data.isna().sum())   

In [7]:
def rows_w_nan(data):
   
    temp = []
    for x in data.columns:
        if data[x].isna().sum() > 0:
            percentage_of_data = data[x].isna().sum()/(list(data.shape)[0])
            print("Row: {} : is made {}% of NaN Values.".format(x, percentage_of_data.round(3)))
            if percentage_of_data > 0.3:
                temp.append(x)
    print(temp)

In [8]:
def howmanyduplicates(data):
    print(data.duplicated().sum())

In [9]:
def rows_w_nan1(data):
   
    for x in data.columns:
        print(x)

# Crashes Data Set

In [10]:
crashes.shape

(484022, 49)

In [11]:
howmanyunique(crashes)

[1, 1, 2, 21, 28, 19, 8, 12, 6, 18, 20, 22, 6, 7, 7, 3, 2, 2, 2, 2, 3, 12, 40, 39, 424, 4, 497, 259, 2, 2, 2, 2, 4, 2, 13, 5, 14, 5, 8, 14, 12, 24, 1, 24, 7, 12, 137, 137, 137]

['CRASH_RECORD_ID', 'RD_NO', 'CRASH_DATE_EST_I', 'CRASH_DATE', 'POSTED_SPEED_LIMIT', 'TRAFFIC_CONTROL_DEVICE', 'DEVICE_CONDITION', 'WEATHER_CONDITION', 'LIGHTING_CONDITION', 'FIRST_CRASH_TYPE', 'TRAFFICWAY_TYPE', 'LANE_CNT', 'ALIGNMENT', 'ROADWAY_SURFACE_COND', 'ROAD_DEFECT', 'REPORT_TYPE', 'CRASH_TYPE', 'INTERSECTION_RELATED_I', 'NOT_RIGHT_OF_WAY_I', 'HIT_AND_RUN_I', 'DAMAGE', 'DATE_POLICE_NOTIFIED', 'PRIM_CONTRIBUTORY_CAUSE', 'SEC_CONTRIBUTORY_CAUSE', 'STREET_NO', 'STREET_DIRECTION', 'STREET_NAME', 'BEAT_OF_OCCURRENCE', 'PHOTOS_TAKEN_I', 'STATEMENTS_TAKEN_I', 'DOORING_I', 'WORK_ZONE_I', 'WORK_ZONE_TYPE', 'WORKERS_PRESENT_I', 'NUM_UNITS', 'MOST_SEVERE_INJURY', 'INJURIES_TOTAL', 'INJURIES_FATAL', 'INJURIES_INCAPACITATING', 'INJURIES_NON_INCAPACITATING', 'INJURIES_REPORTED_NOT_EVIDENT', 'INJURIES_NO_INDICATION',

In [12]:
howmanyduplicates(crashes)

0


In [91]:
rows_w_nan(crashes)

Row: POSTED_SPEED_LIMIT : is made 0.015% of NaN Values.
Row: CRASH_HOUR : is made 0.02% of NaN Values.
[]


In [14]:
# We are going to drop all the columns with a lot of of NaN values, except for location that we can say right ahead that is
# a multicolinearity case, since it has the Latiutude and Longitude together
crashes = crashes.drop(columns=['CRASH_DATE_EST_I', 'LANE_CNT', 'INTERSECTION_RELATED_I',
                                'NOT_RIGHT_OF_WAY_I', 'HIT_AND_RUN_I', 'PHOTOS_TAKEN_I', 
                                'STATEMENTS_TAKEN_I', 'DOORING_I', 'WORK_ZONE_I', 'WORK_ZONE_TYPE',
                                'WORKERS_PRESENT_I', 'LOCATION'])

### Data Manipulation with Missing Values (NaN)

> **RD_NO**

So we can see here that RD_NO and REPORT_TYPE still have a lot of missing data and that can be explained by the website, where they say and I quote:


> " For privacy reasons, this column is blank for recent crashes."

So since the RD_NO is another way to connect the cases between datasets, we can easily discard the whole column, because we still have CRASH_RECORD_ID, that has no missing values

> **REPORT_TYPE**

In [15]:
crashes.REPORT_TYPE.value_counts()

NOT ON SCENE (DESK REPORT)    278124
ON SCENE                      193888
AMENDED                          240
Name: REPORT_TYPE, dtype: int64

By checking the REPORT_TYPE column we realize that it doesn't offer any predictive value to our model, but still might use it for the business understanding part.

> So we decide to drop the RD_NO columns because we have a good substitute and drop the rows where the remaining NaN values are, so we have a cleaner dataset

In [16]:
crashes = crashes.drop(columns=['RD_NO'])

In [17]:
# Now we can drop the remaining rows with NaN values
crashes.dropna(inplace=True)

In [18]:
crashes.isna().sum().sum()

0

### Since "Injuries_Fatal" is important, let's give some more attention to it

We want to transform the column into a Yes or No kind of answer instead of numbers of deads per accident.

In [19]:
crashes.INJURIES_FATAL.value_counts(dropna=False)

0.0    468289
1.0       348
2.0        18
3.0         5
Name: INJURIES_FATAL, dtype: int64

In [20]:
crashes['INJURIES_FATAL'] = np.where(crashes['INJURIES_FATAL']>0, 1, 0)

In [21]:
crashes.INJURIES_FATAL.value_counts(dropna=False)

0    468289
1       371
Name: INJURIES_FATAL, dtype: int64

## Some more data manipulation with useful columns

### WEATHER_CONDITION

In [22]:
crashes.WEATHER_CONDITION.value_counts()

CLEAR                       369648
RAIN                         41293
UNKNOWN                      20530
SNOW                         19487
CLOUDY/OVERCAST              13910
OTHER                         1512
FOG/SMOKE/HAZE                 797
SLEET/HAIL                     692
FREEZING RAIN/DRIZZLE          549
BLOWING SNOW                   154
SEVERE CROSS WIND GATE          86
BLOWING SAND, SOIL, DIRT         2
Name: WEATHER_CONDITION, dtype: int64

In [23]:
crashes.WEATHER_CONDITION = np.where(crashes.WEATHER_CONDITION=='BLOWING SNOW', 'SNOW', crashes.WEATHER_CONDITION)

In [24]:
crashes.WEATHER_CONDITION = np.where(crashes.WEATHER_CONDITION=='FREEZING RAIN/DRIZZLE', 'RAIN', crashes.WEATHER_CONDITION)

In [25]:
crashes.WEATHER_CONDITION = np.where(crashes.WEATHER_CONDITION=='FOG/SMOKE/HAZE', 'OTHER', crashes.WEATHER_CONDITION)

In [26]:
crashes.WEATHER_CONDITION = np.where(crashes.WEATHER_CONDITION=='SLEET/HAIL', 'OTHER', crashes.WEATHER_CONDITION)

In [27]:
crashes.WEATHER_CONDITION = np.where(crashes.WEATHER_CONDITION=='BLOWING SAND, SOIL, DIRT', 'OTHER', crashes.WEATHER_CONDITION)

In [28]:
crashes.WEATHER_CONDITION = np.where(crashes.WEATHER_CONDITION=='SEVERE CROSS WIND GATE', 'OTHER', crashes.WEATHER_CONDITION)

In [29]:
crashes.WEATHER_CONDITION.value_counts()

CLEAR              369648
RAIN                41842
UNKNOWN             20530
SNOW                19641
CLOUDY/OVERCAST     13910
OTHER                3089
Name: WEATHER_CONDITION, dtype: int64

### ALIGNMENT

In [30]:
crashes.ALIGNMENT.value_counts()

STRAIGHT AND LEVEL       457042
STRAIGHT ON GRADE          5775
CURVE, LEVEL               3507
STRAIGHT ON HILLCREST      1440
CURVE ON GRADE              667
CURVE ON HILLCREST          229
Name: ALIGNMENT, dtype: int64

### POSTED_SPEED_LIMIT

In [31]:
crashes.POSTED_SPEED_LIMIT.value_counts()

30    345555
35     32216
25     28456
20     18121
15     16350
10      9874
0       6611
40      4282
5       3551
45      2696
55       360
3        113
50        94
9         89
99        66
39        51
1         33
60        24
2         19
24        16
32        14
33        10
34        10
65        10
6          7
11         5
36         5
70         3
7          2
14         2
12         2
18         2
31         2
26         2
49         1
38         1
22         1
63         1
4          1
23         1
29         1
Name: POSTED_SPEED_LIMIT, dtype: int64

In [32]:
# Let's bin the speed limit in 9 groups, the last one being 45 miles/hour or above
crashes.POSTED_SPEED_LIMIT = pd.cut(crashes.POSTED_SPEED_LIMIT,[0, 5, 10, 15, 20, 25, 30, 35, 40, 45],
                                    precision=0, labels=[0, 1, 2, 3, 4, 5, 6, 7, 8])

In [33]:
crashes.POSTED_SPEED_LIMIT.value_counts()

5    345558
6     32252
4     28474
3     18123
2     16359
1      9972
7      4339
0      3717
8      2696
Name: POSTED_SPEED_LIMIT, dtype: int64

In [34]:
howmanyunique(crashes)

[1, 21, 9, 19, 8, 6, 6, 18, 20, 6, 7, 7, 3, 2, 3, 12, 40, 40, 419, 4, 477, 256, 13, 5, 14, 2, 7, 13, 12, 25, 1, 24, 7, 12, 137, 137]

['CRASH_RECORD_ID', 'CRASH_DATE', 'POSTED_SPEED_LIMIT', 'TRAFFIC_CONTROL_DEVICE', 'DEVICE_CONDITION', 'WEATHER_CONDITION', 'LIGHTING_CONDITION', 'FIRST_CRASH_TYPE', 'TRAFFICWAY_TYPE', 'ALIGNMENT', 'ROADWAY_SURFACE_COND', 'ROAD_DEFECT', 'REPORT_TYPE', 'CRASH_TYPE', 'DAMAGE', 'DATE_POLICE_NOTIFIED', 'PRIM_CONTRIBUTORY_CAUSE', 'SEC_CONTRIBUTORY_CAUSE', 'STREET_NO', 'STREET_DIRECTION', 'STREET_NAME', 'BEAT_OF_OCCURRENCE', 'NUM_UNITS', 'MOST_SEVERE_INJURY', 'INJURIES_TOTAL', 'INJURIES_FATAL', 'INJURIES_INCAPACITATING', 'INJURIES_NON_INCAPACITATING', 'INJURIES_REPORTED_NOT_EVIDENT', 'INJURIES_NO_INDICATION', 'INJURIES_UNKNOWN', 'CRASH_HOUR', 'CRASH_DAY_OF_WEEK', 'CRASH_MONTH', 'LATITUDE', 'LONGITUDE']


In [35]:
crashes.DAMAGE.value_counts()

OVER $1,500      270353
$501 - $1,500    137706
$500 OR LESS      60601
Name: DAMAGE, dtype: int64

In [36]:
crashes.CRASH_DAY_OF_WEEK.value_counts()

6    76212
7    69228
3    67493
5    67214
4    66835
2    65195
1    56483
Name: CRASH_DAY_OF_WEEK, dtype: int64

In [37]:
crashes.CRASH_MONTH.value_counts()

10    45053
12    42845
9     41972
11    41620
8     40634
1     40078
2     40053
7     38888
6     36792
5     35784
3     34476
4     30465
Name: CRASH_MONTH, dtype: int64

In [38]:
crashes.CRASH_HOUR.value_counts()

16    35862
15    35718
17    35351
14    31849
18    29277
13    28993
12    27929
8     25128
11    24209
9     22104
10    21710
19    21359
7     20286
20    16865
21    14982
22    13776
23    11552
6     10548
0      9418
1      7845
2      6799
5      6385
3      5575
4      5140
Name: CRASH_HOUR, dtype: int64

In [39]:
crashes.CRASH_HOUR = pd.cut(crashes.CRASH_HOUR,[0, 6, 12, 18, 24],
                                    precision=0, labels=[0, 1, 2, 3])

In [40]:
crashes.CRASH_HOUR.value_counts()

2    197050
1    141366
3     78534
0     42292
Name: CRASH_HOUR, dtype: int64

In [41]:
crashes.ROADWAY_SURFACE_COND.value_counts()

DRY                348218
WET                 64433
UNKNOWN             31200
SNOW OR SLUSH       19664
ICE                  3810
OTHER                1124
SAND, MUD, DIRT       211
Name: ROADWAY_SURFACE_COND, dtype: int64

In [42]:
crashes.TRAFFICWAY_TYPE.value_counts()

NOT DIVIDED                        209625
DIVIDED - W/MEDIAN (NOT RAISED)     82982
ONE-WAY                             62008
PARKING LOT                         33261
DIVIDED - W/MEDIAN BARRIER          27768
FOUR WAY                            14658
OTHER                               13163
ALLEY                                7731
UNKNOWN                              5111
CENTER TURN LANE                     3992
T-INTERSECTION                       3112
DRIVEWAY                             1622
RAMP                                 1442
UNKNOWN INTERSECTION TYPE             907
Y-INTERSECTION                        373
FIVE POINT, OR MORE                   370
TRAFFIC ROUTE                         270
NOT REPORTED                          118
ROUNDABOUT                             94
L-INTERSECTION                         53
Name: TRAFFICWAY_TYPE, dtype: int64

In [43]:
crashes.MOST_SEVERE_INJURY.value_counts()

NO INDICATION OF INJURY     409702
NONINCAPACITATING INJURY     32671
REPORTED, NOT EVIDENT        18532
INCAPACITATING INJURY         7384
FATAL                          371
Name: MOST_SEVERE_INJURY, dtype: int64

In [44]:
crashes['MOST_SEVERE_INJURY'] = np.where(crashes['MOST_SEVERE_INJURY']=='REPORTED, NOT EVIDENT',
                                         'NONINCAPACITATING INJURY', crashes['MOST_SEVERE_INJURY'])

In [45]:
crashes['MOST_SEVERE_INJURY'] = np.where(crashes['MOST_SEVERE_INJURY']=='FATAL',
                                         'INCAPACITATING INJURY', crashes['MOST_SEVERE_INJURY'])

In [46]:
crashes['MOST_SEVERE_INJURY'] = np.where(crashes['MOST_SEVERE_INJURY']=='INCAPACITATING INJURY',
                                         'INCAPACITATING INJURY/FATAL', crashes['MOST_SEVERE_INJURY'])

In [47]:
crashes.MOST_SEVERE_INJURY.value_counts()

NO INDICATION OF INJURY        409702
NONINCAPACITATING INJURY        51203
INCAPACITATING INJURY/FATAL      7755
Name: MOST_SEVERE_INJURY, dtype: int64

In [48]:
# We could check to see how long the street is OR
# We could create a kind of multi-column based on the other column "TRAFFICWAY_TYPE" OR
# Just group them together by that same column
crashes.STREET_NAME.value_counts()

WESTERN AVE      12896
PULASKI RD       11244
CICERO AVE       10252
ASHLAND AVE      10239
HALSTED ST        8978
                 ...  
ABBOTT AVE           1
LA SALLE             1
57TH BLVD            1
LIGHTFOOT AVE        1
ST JOSEPH AVE        1
Name: STREET_NAME, Length: 1520, dtype: int64

In [49]:
crashes.STREET_DIRECTION.value_counts()

W    166935
S    155627
N    113430
E     32668
Name: STREET_DIRECTION, dtype: int64

In [50]:
crashes

,CRASH_RECORD_ID,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,ALIGNMENT,...,INJURIES_INCAPACITATING,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE
0,4fd0a3e0897b3335b94cd8d5b2d2b350eb691add56c62d...,07/10/2019 05:56:00 PM,6,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,TURNING,ONE-WAY,STRAIGHT AND LEVEL,...,0.0,0.0,0.0,3.0,0.0,2,4,7,41.919664,-87.773288
1,009e9e67203442370272e1a13d6ee51a4155dac65e583d...,06/30/2017 04:00:00 PM,6,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,NOT DIVIDED,STRAIGHT AND LEVEL,...,0.0,0.0,0.0,3.0,0.0,2,6,6,41.741804,-87.740954
2,ee9283eff3a55ac50ee58f3d9528ce1d689b1c4180b4c4...,07/10/2020 10:25:00 AM,5,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,FOUR WAY,STRAIGHT AND LEVEL,...,0.0,0.0,0.0,3.0,0.0,1,6,7,41.773456,-87.585022
3,f8960f698e870ebdc60b521b2a141a5395556bc3704191...,07/11/2020 01:00:00 AM,5,NO CONTROLS,NO CONTROLS,CLEAR,DARKNESS,PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,...,0.0,0.0,0.0,3.0,0.0,0,7,7,41.802119,-87.622115
5,00e47f189660cd8ba1e85fc63061bf1d8465184393f134...,03/21/2019 10:50:00 PM,5,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",TURNING,NOT DIVIDED,STRAIGHT AND LEVEL,...,0.0,0.0,0.0,2.0,0.0,3,5,3,41.741804,-87.740954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484017,f37d01cdad7c3e9fe01562aa11857405cc2ea38738d8b6...,01/19/2021 07:30:00 AM,2,NO CONTROLS,NO CONTROLS,CLOUDY/OVERCAST,DAYLIGHT,FIXED OBJECT,PARKING LOT,STRAIGHT AND LEVEL,...,0.0,0.0,0.0,2.0,0.0,1,3,1,41.877790,-87.636488
484018,f1dba052d8fc8c80d3d693296ff8e0d7cc71d5929677b0...,01/20/2021 05:50:00 PM,5,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,NOT DIVIDED,STRAIGHT AND LEVEL,...,0.0,0.0,0.0,2.0,0.0,2,4,1,41.800697,-87.706358
484019,f2962f241a302417913e1b7465e8ae37e6f3161adaa0cf...,01/20/2021 04:15:00 PM,5,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,NOT DIVIDED,STRAIGHT AND LEVEL,...,0.0,0.0,0.0,2.0,0.0,2,4,1,41.924709,-87.717371
484020,f361f7362d783dc327e5e6b7d2d9bc4bd942284391ad36...,01/20/2021 03:20:00 PM,5,STOP SIGN/FLASHER,NO CONTROLS,CLOUDY/OVERCAST,DAYLIGHT,ANGLE,NOT DIVIDED,STRAIGHT AND LEVEL,...,0.0,0.0,0.0,4.0,0.0,2,4,1,41.802871,-87.684416


### Working with Date-Time Columns

- Can we explore and see if holidays play a part on accidents?

In [51]:
# CRASH_DATE is a string
crashes['CRASH_DATE'][0]

'07/10/2019 05:56:00 PM'

In [52]:
# We can check that here
type(crashes['CRASH_DATE'][0])

str

In [53]:
# We create a new column Date-Time with the previous column
crashes['DATE_ACCIDENT']= pd.to_datetime(crashes['CRASH_DATE'], format='%m/%d/%Y %I:%M:%S %p')

In [54]:
# We confirm that it worked
type(crashes.DATE_ACCIDENT[0])

pandas._libs.tslibs.timestamps.Timestamp

In [55]:
# Our earliest and latest date
print("Earliest Date: {} \nLatest Date: {}".format(crashes['DATE_ACCIDENT'].min(), crashes['DATE_ACCIDENT'].max()))

Earliest Date: 2013-03-03 16:48:00 
Latest Date: 2021-03-09 23:10:00


In [56]:
holidays = pd.tseries.holiday.USFederalHolidayCalendar().holidays(start='2012', end='2022').to_pydatetime()

In [57]:
holidays_date = [holiday.date() for holiday in holidays]


In [58]:
holidays_date[:3]

[datetime.date(2012, 1, 2),
 datetime.date(2012, 1, 16),
 datetime.date(2012, 2, 20)]

In [59]:
def isitaholiday(date):
    ''' super useful function'''
    if date.date() in holidays_date:
        return 1
    else: 
        return 0

In [60]:
isitaholiday(crashes['DATE_ACCIDENT'][100])

0

In [61]:
holidays[1].date()

datetime.date(2012, 1, 16)

In [62]:
crashes['IS_A_HOLIDAY'] = crashes['DATE_ACCIDENT'].apply(isitaholiday)

In [63]:
crashes['HOLIDAY_NAME'] = crashes['DATE_ACCIDENT'].apply(isitaholiday)

In [64]:
crashes['IS_A_HOLIDAY'].sum()

11064

In [65]:
crashes['DATE_ACCIDENT'][0].date()

datetime.date(2019, 7, 10)

In [66]:
crashes.drop(['CRASH_DATE'], axis = 1, inplace = True)

In [67]:
crashes.head(3)

,CRASH_RECORD_ID,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,ALIGNMENT,ROADWAY_SURFACE_COND,...,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,DATE_ACCIDENT,IS_A_HOLIDAY,HOLIDAY_NAME
0,4fd0a3e0897b3335b94cd8d5b2d2b350eb691add56c62d...,6,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,TURNING,ONE-WAY,STRAIGHT AND LEVEL,DRY,...,3.0,0.0,2,4,7,41.919664,-87.773288,2019-07-10 17:56:00,0,0
1,009e9e67203442370272e1a13d6ee51a4155dac65e583d...,6,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,...,3.0,0.0,2,6,6,41.741804,-87.740954,2017-06-30 16:00:00,0,0
2,ee9283eff3a55ac50ee58f3d9528ce1d689b1c4180b4c4...,5,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,FOUR WAY,STRAIGHT AND LEVEL,DRY,...,3.0,0.0,1,6,7,41.773456,-87.585022,2020-07-10 10:25:00,0,0


In [68]:
crashes_holiday = crashes[crashes['IS_A_HOLIDAY']==1]

In [69]:
crashes_holiday.head(3)

,CRASH_RECORD_ID,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,ALIGNMENT,ROADWAY_SURFACE_COND,...,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,DATE_ACCIDENT,IS_A_HOLIDAY,HOLIDAY_NAME
155,3bfec94191bdc869600d2c429c45a68871eb9733ce7ec6...,4,NO CONTROLS,NO CONTROLS,SNOW,DAYLIGHT,PARKED MOTOR VEHICLE,NOT DIVIDED,STRAIGHT AND LEVEL,SNOW OR SLUSH,...,1.0,0.0,1,6,11,41.932213,-87.653694,2017-11-10 12:45:00,1,1
168,43292e98b1ba60cb4c6efeb987a67a1ec0291acea4357e...,7,NO CONTROLS,OTHER,RAIN,"DARKNESS, LIGHTED ROAD",TURNING,NOT DIVIDED,STRAIGHT AND LEVEL,WET,...,4.0,0.0,NaN,2,2,41.880660,-87.740728,2018-02-19 00:25:00,1,1
234,5e45d4537c4d0bd12bfa11a7f75526f4fe0d61c68a0ec1...,1,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,PARKING LOT,STRAIGHT AND LEVEL,WET,...,1.0,0.0,0,3,1,41.879714,-87.774496,2019-01-01 02:40:00,1,1


### **To Delete?**
- 'FIRST_CRASH_TYPE'
- 'INJURIES_INCAPACITATING' 
- 'INJURIES_NON_INCAPACITATING'
- 'INJURIES_REPORTED_NOT_EVIDENT'
- 'INJURIES_TOTAL'
- 'MOST_SEVERE_INJURY'
- 'NUM_UNITS'
- 'BEAT_OF_OCCURRENCE'
- 'DATE_POLICE_NOTIFIED'
- 'PRIM_CONTRIBUTORY_CAUSE'
- 'SEC_CONTRIBUTORY_CAUSE'

# People Data Set

In [70]:
people.shape

(1069147, 30)

In [71]:
howmanyunique(people)

[1, 6, 45, 45, 43, 51, 11, 335, 49, 364, 3, 108, 72, 61, 19, 7, 5, 5, 187, 175, 96, 20, 14, 12, 23, 4, 8, 4, 30, 2]

['PERSON_ID', 'PERSON_TYPE', 'CRASH_RECORD_ID', 'RD_NO', 'VEHICLE_ID', 'CRASH_DATE', 'SEAT_NO', 'CITY', 'STATE', 'ZIPCODE', 'SEX', 'AGE', 'DRIVERS_LICENSE_STATE', 'DRIVERS_LICENSE_CLASS', 'SAFETY_EQUIPMENT', 'AIRBAG_DEPLOYED', 'EJECTION', 'INJURY_CLASSIFICATION', 'HOSPITAL', 'EMS_AGENCY', 'EMS_RUN_NO', 'DRIVER_ACTION', 'DRIVER_VISION', 'PHYSICAL_CONDITION', 'PEDPEDAL_ACTION', 'PEDPEDAL_VISIBILITY', 'PEDPEDAL_LOCATION', 'BAC_RESULT', 'BAC_RESULT VALUE', 'CELL_PHONE_USE']


In [72]:
howmanyduplicates(people)

0


In [73]:
rows_w_nan(people)

Row: RD_NO : is made 0.007% of NaN Values.
Row: VEHICLE_ID : is made 0.02% of NaN Values.
Row: SEAT_NO : is made 0.796% of NaN Values.
Row: CITY : is made 0.261% of NaN Values.
Row: STATE : is made 0.253% of NaN Values.
Row: ZIPCODE : is made 0.325% of NaN Values.
Row: SEX : is made 0.015% of NaN Values.
Row: AGE : is made 0.285% of NaN Values.
Row: DRIVERS_LICENSE_STATE : is made 0.407% of NaN Values.
Row: DRIVERS_LICENSE_CLASS : is made 0.485% of NaN Values.
Row: SAFETY_EQUIPMENT : is made 0.003% of NaN Values.
Row: AIRBAG_DEPLOYED : is made 0.019% of NaN Values.
Row: EJECTION : is made 0.012% of NaN Values.
Row: INJURY_CLASSIFICATION : is made 0.001% of NaN Values.
Row: HOSPITAL : is made 0.818% of NaN Values.
Row: EMS_AGENCY : is made 0.884% of NaN Values.
Row: EMS_RUN_NO : is made 0.981% of NaN Values.
Row: DRIVER_ACTION : is made 0.206% of NaN Values.
Row: DRIVER_VISION : is made 0.207% of NaN Values.
Row: PHYSICAL_CONDITION : is made 0.206% of NaN Values.
Row: PEDPEDAL_ACTION : 

In [74]:
people = people.drop(columns=['CELL_PHONE_USE', 'BAC_RESULT VALUE', 'PEDPEDAL_LOCATION', 'PEDPEDAL_VISIBILITY',
                              'EMS_RUN_NO', 'EMS_AGENCY', 'HOSPITAL', 'DRIVERS_LICENSE_CLASS', 
                              'DRIVERS_LICENSE_STATE', 'ZIPCODE', 'SEAT_NO', 'PEDPEDAL_ACTION'])

In [75]:
#people.dropna(inplace=True)

In [76]:
people.isna().sum()

PERSON_ID                     0
PERSON_TYPE                   0
CRASH_RECORD_ID               0
RD_NO                      7228
VEHICLE_ID                21151
CRASH_DATE                    0
CITY                     278763
STATE                    269963
SEX                       15827
AGE                      304698
SAFETY_EQUIPMENT           3170
AIRBAG_DEPLOYED           20244
EJECTION                  13192
INJURY_CLASSIFICATION       565
DRIVER_ACTION            220503
DRIVER_VISION            220783
PHYSICAL_CONDITION       219892
BAC_RESULT               219396
dtype: int64

In [77]:
people.INJURY_CLASSIFICATION.value_counts()

NO INDICATION OF INJURY     982663
NONINCAPACITATING INJURY     47604
REPORTED, NOT EVIDENT        28368
INCAPACITATING INJURY         9400
FATAL                          547
Name: INJURY_CLASSIFICATION, dtype: int64

In [78]:
people.AIRBAG_DEPLOYED.value_counts()

DID NOT DEPLOY                            632622
DEPLOYMENT UNKNOWN                        202892
NOT APPLICABLE                            152987
DEPLOYED, FRONT                            32011
DEPLOYED, COMBINATION                      19738
DEPLOYED, SIDE                              8230
DEPLOYED OTHER (KNEE, AIR, BELT, ETC.)       423
Name: AIRBAG_DEPLOYED, dtype: int64

In [79]:
# Useless?
people.EJECTION.value_counts()

NONE                  992659
UNKNOWN                57641
TOTALLY EJECTED         3955
PARTIALLY EJECTED       1034
TRAPPED/EXTRICATED       666
Name: EJECTION, dtype: int64

In [80]:
people.PHYSICAL_CONDITION.value_counts()

NORMAL                          570812
UNKNOWN                         262391
IMPAIRED - ALCOHOL                4149
REMOVED BY EMS                    3308
OTHER                             2352
FATIGUED/ASLEEP                   2252
EMOTIONAL                         1755
ILLNESS/FAINTED                    766
HAD BEEN DRINKING                  681
IMPAIRED - DRUGS                   498
IMPAIRED - ALCOHOL AND DRUGS       184
MEDICATED                          107
Name: PHYSICAL_CONDITION, dtype: int64

In [81]:
people.PERSON_TYPE.value_counts()

DRIVER                 829972
PASSENGER              218450
PEDESTRIAN              12359
BICYCLE                  7214
NON-MOTOR VEHICLE         951
NON-CONTACT VEHICLE       201
Name: PERSON_TYPE, dtype: int64

# Vehicles Data Set

**SEAT_NO**
- Code for seating position of motor vehicle occupant: 1= driver, 2= center front, 3 = front passenger, 4 = second row left, 5 = second row center, 6 = second row right, 7 = enclosed passengers, 8 = exposed passengers, 9= unknown position, 10 = third row left, 11 = third row center, 12 = third row right

In [82]:
vehicles.shape

(988131, 72)

In [83]:
howmanyunique(vehicles)

[1, 17, 17, 40, 17, 9, 22, 1, 2, 149, 540, 51, 68, 17, 21, 25, 9, 28, 2, 2, 25, 2, 130, 119, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 14, 1, 33, 11, 11, 4, 40, 37, 39, 59, 2, 3, 8, 3, 3, 1, 3, 1, 3, 3, 9, 2, 3, 3, 26, 4, 39, 11, 8, 9, 6, 2, 2, 7]

['CRASH_UNIT_ID', 'CRASH_RECORD_ID', 'RD_NO', 'CRASH_DATE', 'UNIT_NO', 'UNIT_TYPE', 'NUM_PASSENGERS', 'VEHICLE_ID', 'CMRC_VEH_I', 'MAKE', 'MODEL', 'LIC_PLATE_STATE', 'VEHICLE_YEAR', 'VEHICLE_DEFECT', 'VEHICLE_TYPE', 'VEHICLE_USE', 'TRAVEL_DIRECTION', 'MANEUVER', 'TOWED_I', 'FIRE_I', 'OCCUPANT_CNT', 'EXCEED_SPEED_LIMIT_I', 'TOWED_BY', 'TOWED_TO', 'AREA_00_I', 'AREA_01_I', 'AREA_02_I', 'AREA_03_I', 'AREA_04_I', 'AREA_05_I', 'AREA_06_I', 'AREA_07_I', 'AREA_08_I', 'AREA_09_I', 'AREA_10_I', 'AREA_11_I', 'AREA_12_I', 'AREA_99_I', 'FIRST_CONTACT_POINT', 'CMV_ID', 'USDOT_NO', 'CCMC_NO', 'ILCC_NO', 'COMMERCIAL_SRC', 'GVWR', 'CARRIER_NAME', 'CARRIER_STATE', 'CARRIER_CITY', 'HAZMAT_PLACARDS_I', 'HAZMAT_NAME', 'UN_NO', 'HAZMAT_PRESENT_I', 'HAZMAT_REPORT

In [84]:
howmanyduplicates(vehicles)

0


In [90]:
rows_w_nan(vehicles)

[]


In [86]:
vehicles = vehicles.drop(columns=['NUM_PASSENGERS', 'CMRC_VEH_I', 'TOWED_I', 'FIRE_I', 'EXCEED_SPEED_LIMIT_I', 
                                  'TOWED_BY', 'TOWED_TO', 'AREA_00_I', 'AREA_01_I', 'AREA_02_I', 'AREA_03_I', 
                                  'AREA_04_I', 'AREA_05_I', 'AREA_06_I', 'AREA_07_I', 'AREA_08_I', 'AREA_09_I', 
                                  'AREA_10_I', 'AREA_11_I', 'AREA_12_I', 'AREA_99_I', 'CMV_ID', 'USDOT_NO', 'CCMC_NO', 
                                  'ILCC_NO', 'COMMERCIAL_SRC', 'GVWR', 'CARRIER_NAME', 'CARRIER_STATE', 'CARRIER_CITY',
                                  'HAZMAT_PLACARDS_I', 'HAZMAT_NAME', 'UN_NO', 'HAZMAT_PRESENT_I', 'HAZMAT_REPORT_I',
                                  'HAZMAT_REPORT_NO', 'MCS_REPORT_I', 'MCS_REPORT_NO', 'HAZMAT_VIO_CAUSE_CRASH_I',
                                  'MCS_VIO_CAUSE_CRASH_I', 'IDOT_PERMIT_NO', 'WIDE_LOAD_I', 'TRAILER1_WIDTH',
                                  'TRAILER2_WIDTH', 'TRAILER1_LENGTH', 'TRAILER2_LENGTH', 'TOTAL_VEHICLE_LENGTH',
                                  'AXLE_CNT', 'VEHICLE_CONFIG', 'CARGO_BODY_TYPE', 'LOAD_TYPE', 'HAZMAT_OUT_OF_SERVICE_I',
                                  'MCS_OUT_OF_SERVICE_I', 'HAZMAT_CLASS', 'LIC_PLATE_STATE'])

In [87]:
vehicles.dropna(inplace=True)

In [88]:
vehicles.isna().sum()

CRASH_UNIT_ID          0
CRASH_RECORD_ID        0
RD_NO                  0
CRASH_DATE             0
UNIT_NO                0
UNIT_TYPE              0
VEHICLE_ID             0
MAKE                   0
MODEL                  0
VEHICLE_YEAR           0
VEHICLE_DEFECT         0
VEHICLE_TYPE           0
VEHICLE_USE            0
TRAVEL_DIRECTION       0
MANEUVER               0
OCCUPANT_CNT           0
FIRST_CONTACT_POINT    0
dtype: int64

# Notes:

In [89]:
# new = old[['A', 'C', 'D']]

1) Chicago Car Crashes (Links to an external site.)
Note this links also to Vehicle Data (Links to an external site.) and to Driver/Passenger Data (Links to an external site.).

Build a classifier to predict the primary contributory cause of a car accident, given information about the car,
the people in the car,the road conditions etc. You might imagine your audience as a Vehicle Safety Board who's
interested in reducing traffic accidents, or as the City of Chicago who's interested in becoming aware of any
interesting patterns. Note that there is a multi-class classification problem. You will almost certainly want to bin 
or trim or otherwise limit the number of target categories on which you ultimately predict. Note e.g. that some
primary contributory causes have very few samples.

This criterion is linked to a Learning OutcomeBusiness Understanding

Explains the project's real-world value for a specific stakeholder

This criterion is linked to a Learning OutcomeData Understanding

Explicitly relates data's source and properties to real-world problem

This criterion is linked to a Learning OutcomeData Preparation

Data preparation is reproducible and well-documented with valid justifications

Modeling

Model development is iterative and documented with valid justifications

This criterion is linked to a Learning OutcomeClassification Results

Explains how well the project solves the real-world problem

This criterion is linked to a Learning OutcomeVisualization

Includes three polished visualizations of relevant findings

GitHub Repository

Repository uses all "best practices" for README, structure, and commits

Presentation Content

Presentation clearly shows stakeholders how well the project meets their needs

This criterion is linked to a Learning OutcomeSlide Style

Slides have a fully professional style

This criterion is linked to a Learning OutcomePresentation Delivery

Presentation engagingly and clearly delivers the intended content

This criterion is linked to a Learning OutcomeAnswers to Questions

Answers are fully clear and appropriate